In [103]:
import pandas as pd
import numpy as np

In [106]:
items = pd.read_csv("Csv_limpios/steam_items_limpio.csv")
reviews = pd.read_csv("Csv_limpios/steam_reviews_limpio.csv")
steam_games = pd.read_csv("Csv_limpios/steam_games_limpio.csv")
steam_games = steam_games.drop_duplicates()

#Pasamos la columna a tipo STR
steam_games['genres'] = steam_games['genres'].astype(str)
steam_games['title'] = steam_games['title'].astype(str)

#Agrupamos los juegos con todos sus generos

steam_games_groupby = steam_games.groupby(['title', "id"])['genres'].apply(lambda x: ' '.join(x)).reset_index()

steam_games_groupby = steam_games_groupby.dropna()

steam_games_groupby.to_csv("Csv_api/funcionML.csv", index=False)

steam_games_groupby.isna().mean()

,title,id,genres
0,! That Bastard Is Trying To Steal Our Gold !,449940,Action Adventure Casual Indie
1,"""Barely Attuned Magic Thingy"" Staff",308163,Action Free to Play Indie Massively Multiplaye...
2,"""Glow Ball"" - The billiard puzzle game",388390,Casual Indie Sports Strategy
3,"""Just Another Day"" - Seduce Me Otome CD",454790,Casual Simulation
4,"""Lethargic Sentience"" Wand",308164,Action Free to Play Indie Massively Multiplaye...
...,...,...,...
30034,（尘沙惑设定集）Lost in Secular Love - Concept Design ...,541220,Adventure Casual Indie Simulation
30035,４人打ちアクション麻雀 / ACTION MAHJONG,575810,Action Casual Indie Early Access
30036,＜/reality＞,562280,Adventure Indie
30037,＜/reality＞ Original Soundtrack,626850,Adventure Indie


In [99]:
#Importamos las librerias necesarias para el modelo de recomendacion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#Iniciamos el countvectorizer en una variable para pasar los generos a numero
vectorizer = CountVectorizer()

#Pasamos los generos a numero
genres_vectorized = vectorizer.fit_transform(steam_games_groupby["genres"]).toarray()

In [100]:
#Calculamos la similitud del coseno
cosine_sim = cosine_similarity(genres_vectorized)

In [101]:
#Creamos la funcion
def recomendacion_juego(id):
    #Extraemos el indice del juego ingresado
    idx = steam_games_groupby[steam_games_groupby["id"] == id].index[0]

    #Comparamos la similitud del coseno entre el juego ingresado y los demas juegos
    evaluacion = cosine_sim[idx]
    #Selecciono los 5 juegos mas parecidos segun la similitud del coseno
    juegos = sorted(list(enumerate(evaluacion)), reverse=True, key=lambda x: x[1])[1:6]

    juegos_recomendados = [steam_games_groupby.iloc[i[0]]['title'] for i in juegos]

    return juegos_recomendados

recomendacion_juego(462213)

['Dignified Fantasy Music Vol.2',
 'RPG Maker 2003',
 'RPG Maker MV',
 'RPG Maker MV - 8-bit Perfect Collection',
 'RPG Maker MV - Animations Collection I: Quintessence']

In [102]:
steam_games_groupby

,title,id,genres
0,! That Bastard Is Trying To Steal Our Gold !,449940,Action Adventure Casual Indie
1,"""Barely Attuned Magic Thingy"" Staff",308163,Action Free to Play Indie Massively Multiplaye...
2,"""Glow Ball"" - The billiard puzzle game",388390,Casual Indie Sports Strategy
3,"""Just Another Day"" - Seduce Me Otome CD",454790,Casual Simulation
4,"""Lethargic Sentience"" Wand",308164,Action Free to Play Indie Massively Multiplaye...
...,...,...,...
30034,（尘沙惑设定集）Lost in Secular Love - Concept Design ...,541220,Adventure Casual Indie Simulation
30035,４人打ちアクション麻雀 / ACTION MAHJONG,575810,Action Casual Indie Early Access
30036,＜/reality＞,562280,Adventure Indie
30037,＜/reality＞ Original Soundtrack,626850,Adventure Indie
